In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os
import warnings

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
warnings.filterwarnings('ignore')

from utils import *

In [0]:
# # todo standardise pipeline
# import os
# import pandas as pd
# import numpy as np
# import joblib  # For efficient saving and loading

# # Define paths
# BASE_DIR = "data"

# def ensure_dir(directory):
#     """Ensure directory exists."""
#     if not os.path.exists(directory):
#         os.makedirs(directory)

# def load_or_compute(filepath, func, *args, **kwargs):
#     """
#     Check if the processed file exists. 
#     If yes, load and return the data; otherwise, compute, save, and return.
#     """
#     if os.path.exists(filepath):
#         print(f"Loading existing data: {filepath}")
#         return joblib.load(filepath)
#     else:
#         print(f"Processing and saving data: {filepath}")
#         data = func(*args, **kwargs)
#         joblib.dump(data, filepath)
#         return data

# # Define processing steps
# def load_data(filepath):
#     """Load raw dataset."""
#     return pd.read_csv(filepath)

# def log_transform(df, cols):
#     """Apply log transformation to specified columns."""
#     df_transformed = df.copy()
#     for col in cols:
#         df_transformed[col] = np.log1p(df_transformed[col])  # log(1 + x) to avoid log(0) issues
#     return df_transformed

# def scale_data(df):
#     """Min-max scale all numeric columns."""
#     from sklearn.preprocessing import MinMaxScaler
#     scaler = MinMaxScaler()
#     df_scaled = df.copy()
#     df_scaled[df.select_dtypes(include=[np.number]).columns] = scaler.fit_transform(df_scaled.select_dtypes(include=[np.number]))
#     return df_scaled

# def remove_high_corr(df, threshold=0.9):
#     """Remove one of two highly correlated columns above a threshold."""
#     corr_matrix = df.corr().abs()
#     upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
#     to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
#     return df.drop(columns=to_drop)

# # Process Pipeline
# def process_pipeline(raw_data_path):
#     """Execute the processing pipeline step-by-step, saving intermediate results."""
#     ensure_dir(BASE_DIR)

#     # Step 1: Load data
#     data = load_or_compute(f"{BASE_DIR}/raw_data.pkl", load_data, raw_data_path)

#     # Step 2: Log transformation
#     log_cols = ["feature1", "feature2"]  # Replace with actual numeric columns needing log transformation
#     data_log = load_or_compute(f"{BASE_DIR}/log_transformed.pkl", log_transform, data, log_cols)

#     # Step 3: Scaling
#     data_scaled = load_or_compute(f"{BASE_DIR}/scaled.pkl", scale_data, data_log)

#     # Step 4: Remove highly correlated columns
#     final_data = load_or_compute(f"{BASE_DIR}/cleaned.pkl", remove_high_corr, data_scaled)

#     return final_data

# # Run pipeline
# processed_df = process_pipeline("path/to/your/dataset.csv")


# absolute counts 

**absolute counts**


In [0]:
df_counts = pd.read_csv('../data/taxa_data_counts.csv', sep=',', header=0, index_col=0)
df_counts.fillna(0, inplace=True)
df_counts.columns = df_counts.columns.str.replace('[', '').str.replace(']', '')
df_counts

In [0]:
plot_data_distribution(df_counts, 'taxa counts')

**to genus**

In [0]:
df_genus_counts = df_counts.groupby(df_counts.columns.str.split(' ').str[0], axis=1).sum()
print(df_genus_counts.shape)
plot_data_distribution(df_genus_counts, 'genus counts')

**log transformation**

In [0]:
pseudo_count = 10 ** np.log10(df_genus_counts[df_genus_counts > 0].min().min()**2 / df_genus_counts.max().max())
print(pseudo_count)
df_genus_counts_log = df_genus_counts.applymap(lambda x: np.log10(x + pseudo_count))

plot_data_distribution(df_genus_counts_log, 'Log10 Genus-level counts')

**rescale to 0-1**

In [0]:
df_genus_counts_log_scaled = (df_genus_counts_log - df_genus_counts_log.min().min())/(df_genus_counts_log.max().max() - df_genus_counts_log.min().min())
print(df_genus_counts_log_scaled.min().min(), df_genus_counts_log_scaled.max().max())

plot_data_distribution(df_genus_counts_log_scaled, 'Rescaled_log10 Genus-level counts')

In [0]:
df_genus_counts_log_scaled.to_csv('../data/genus_counts_log_scaled.csv', sep='\t', header=True, index=True)

In [0]:
df_genus_counts_log_reduced = feature_reduction_pipeline(df_genus_counts_log, variance_threshold=0.01, correlation_threshold=0.8, viz_corr_clusters=True, fname='df_genus_counts_log_reduced')

In [0]:
from utils import *
df_genus_counts_log_scaled_reduced = feature_reduction_pipeline(df_genus_counts_log_scaled, variance_threshold=0.01, correlation_threshold=0.8, viz_corr_clusters=True,fname='df_genus_counts_log_scaled_reduced')


In [0]:
plot_data_distribution(df_genus_counts_log_scaled_reduced, 'Log10 Genus-level counts after feature reduction')

In [0]:
df_genus_counts_log_scaled_reduced.to_csv('../data/genus_counts_log_scaled_reduced.csv', sep='\t', header=True, index=True)

# log abundance


In [0]:

df_log = pd.read_csv('../data/processed_log_abundance.csv', sep='\t', header=0, index_col=0)
df_log

In [0]:
lst_taxa = list(df_log.columns)

In [0]:
corr_matrix_log = df_log.corr().abs()

In [0]:
print(np.__version__)

1.26.4 (13/02/2025) Qibo - GPU - small

In [0]:
upper_log = corr_matrix_log.where(np.triu(np.ones(corr_matrix_log.shape), k=1).astype(bool))
to_drop_log = [column for column in upper_log.columns if any(upper_log[column] > 0.8)]
len(to_drop_log)

In [0]:
dct_corr_to_drop = {}
for column in upper_log.columns:
    if any(upper_log[column] > 0.8):
        dct_corr_to_drop[column] = list(upper_log.index[upper_log[column] > 0.8])
dct_corr_to_drop

In [0]:
df_log_drop08 = df_log.drop(columns=to_drop_log, axis=1, inplace=False)
print(df_log_drop08.shape)
df_log_drop08.max().max(), df_log_drop08.min().min()

In [0]:
df_log_drop08_scaled = (df_log_drop08 - df_log_drop08.min().min()) /(df_log_drop08.max().max() - df_log_drop08.min().min())
df_log_drop08_scaled.max().max(), df_log_drop08_scaled.min().min()

In [0]:
df_log_drop08_scaled.to_csv('../data/processed_log_drop08_scaled.csv', sep='\t', header=True, index=True)

In [0]:
to_drop_log = [column for column in upper_log.columns if any(upper_log[column] > 0.9)]
len(to_drop_log)

# original abundance data

In [0]:
df_abundance = pd.read_csv("../data/taxa_data.txt", sep='\t', header=0, index_col=0)
df_abundance.columns = df_abundance.columns.str.replace('[', '').str.replace(']', '')
df_abundance = df_abundance.fillna(0)
print(df_abundance.shape)
df_abundance

In [0]:
df_abundance = df_abundance.loc[:, (df_abundance != 0).any(axis=0)]
df_abundance.shape

In [0]:
# remove highly associated taxa
corr_matrix = df_abundance.corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.9)]
len(to_drop)

In [0]:
!pip install matplotlib-venn

In [0]:
import matplotlib.pyplot as plt
from matplotlib_venn import venn2

venn2([set(to_drop), set(to_drop_log)], ('to_drop', 'to_drop_log'))
plt.show()

In [0]:
threshold = 0.8
to_drop_log = [column for column in upper_log.columns if any(upper_log[column] > threshold)]
to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
venn2([set(to_drop), set(to_drop_log)], ('to_drop', 'to_drop_log'))


In [0]:
df_abundance_reduced = df_abundance.drop(columns=to_drop, axis=1, inplace=False)


In [0]:
df_abundance.max().max()

In [0]:
df_abundance.to_csv('../data/processed_abundance.csv', sep='\t', header=True, index=True)

In [0]:
10 ** np.log10(df_abundance[df_abundance > 0].min().min() **2 / df_abundance.max().max())

In [0]:
df_log = df_abundance.applymap(lambda x: np.log10(x + 9e-12))
print(df_log.shape)
df_log.to_csv('../data/processed_log_abundance.csv', sep='\t', header=True, index=True)
df_log

In [0]:
df_meta = pd.read_csv("../data/metadata.txt", sep='\t', header=0, index_col=0)
df_meta

In [0]:
df_abundance.index.equals(df_meta.index)


In [0]:
# Calculate proportion of healthy individuals by age
df_meta['healthy'] = df_meta['disease'] == 'healthy'
proportion_healthy_df = df_meta.groupby('age')['healthy'].mean().reset_index(name='proportion_healthy')
proportion_healthy_df

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt

# Plot histogram
sns.histplot(data=df_meta, x='age', bins=int(df_meta['age'].max() + 1), hue='age_category')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Distribution of Age')

In [0]:
# Plot histogram
sns.histplot(data=df_meta[df_meta['age'] > 0], x='age', bins=int(df_meta['age'].max()), hue='gender', multiple='stack')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Distribution of Age (age > 0)')

# Create a secondary y-axis
ax1 = plt.gca()
ax2 = ax1.twinx()

# Add curve for proportion of healthy individuals
sns.lineplot(data=proportion_healthy_df[proportion_healthy_df['age'].isin(df_meta['age'][df_meta['age'] > 0])], x='age', y='proportion_healthy', color='green', label='Proportion Healthy', ax=ax2)
ax2.set_ylabel('Proportion Healthy')

plt.legend(loc='upper center')
plt.show()

In [0]:
# remove age = 0 and remove disease 
df_meta_healthy = df_meta[df_meta['healthy']]
df_meta_healthy_not_newborn = df_meta_healthy[df_meta_healthy['age'] > 0]

In [0]:
df_meta_healthy_not_newborn.groupby('age')['healthy'].mean().reset_index(name='proportion_healthy')

In [0]:
df_meta_healthy_not_newborn.groupby('age')['healthy'].mean().reset_index(name='proportion_healthy').proportion_healthy.unique()

In [0]:
# Plot histogram
sns.histplot(data=df_meta_healthy_not_newborn, x='age', bins=int(df_meta['age'].max()-1), hue='gender', multiple='stack')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.title('Distribution of Age for Healthy age > 0 Samples')

# Create a secondary y-axis
ax1 = plt.gca()
ax2 = ax1.twinx()

plt.legend(loc='upper center')

;

In [0]:
df_meta_healthy_not_newborn[['age']].to_csv('../data/age.csv', sep='\t', header=False, index=True)

## disease pred data

In [0]:
df_disease = df_meta['disease'].str.get_dummies(sep=';')
df_disease


In [0]:
plt.figure(figsize=(18, 5))
sum_dis = df_disease.sum(axis=0)
print(sum_dis.sort_values(ascending=False))
sns.barplot(x=sum_dis.index, y=sum_dis.values)
plt.xticks(rotation=60, fontsize=10, ha='right');



In [0]:
df_disease[['T2D']].to_csv('../DeepMicro/data/T2D.csv', sep=',', header=False, index=False)

In [0]:
df_disease['sample_id'] = df_meta['sample_id']
df_data = pd.merge(df_disease[['sample_id', 'T2D']], df_abundance, on="sample_id", how='inner')
df_data

In [0]:
df_data['T2D'].value_counts()

In [0]:
df_data['target'] = df_data['T2D']

In [0]:
df_data.drop(columns=['sample_id', 'T2D'], inplace=True)
df_data

In [0]:
# df_data.to_csv('../data/processed_log_abundance_t2d.csv', sep='\t', header=True, index=False)

In [0]:
df_data.isna().any(axis=1).sum()

# genus level abundance

In [0]:
from utils_data import *

In [0]:
plot_data_distribution(df_abundance, 'Species-level Abundance')

In [0]:
# Group columns by their starting word and sum their values
df_genus = df_abundance.groupby(df_abundance.columns.str.split(' ').str[0], axis=1).sum()

df_genus

In [0]:
plot_data_distribution(df_genus, 'Genus-level Abundance')

In [0]:
# corr_matrix_genus = df_genus.corr().abs()
# upper = corr_matrix_genus.where(np.triu(np.ones(corr_matrix_genus.shape), k=1).astype(bool))
# to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
# print(len(to_drop))
# to_drop

In [0]:
10 ** np.log10(df_genus[df_genus > 0].min().min()**2 / df_genus.max().max())

In [0]:
df_genus_log = df_genus.applymap(lambda x: np.log10(x + 9e-12))

In [0]:
plot_data_distribution(df_genus_log, 'Log10 Genus-level Abundance')

In [0]:
corr_matrix_genus = df_genus_log.corr().abs()
upper = corr_matrix_genus.where(np.triu(np.ones(corr_matrix_genus.shape), k=1).astype(bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print(len(to_drop))
to_drop

In [0]:
df_genus_log_drop08 = df_genus_log.drop(columns=to_drop, axis=1, inplace=False)
print(df_genus_log_drop08.shape)
print(df_genus_log_drop08.max().max(), df_genus_log_drop08.min().min())

In [0]:

plot_data_distribution(df_genus_log_drop08, 'Log10 non-correlated Genus-level Abundance')

In [0]:
df_genus_log_drop08_scaled = (df_genus_log - df_genus_log_drop08.min().min())/(df_genus_log_drop08.max().max() - df_genus_log_drop08.min().min())
print(df_genus_log_drop08_scaled.min().min(), df_genus_log_drop08_scaled.max().max())


In [0]:
plot_data_distribution(df_genus_log_drop08_scaled, 'Scaled Log10 non-correlated Genus-level Abundance')

In [0]:
df_genus_log_drop08_scaled

In [0]:
import os

if os.path.exists('../data/processed_genus_log_drop08_scaled.csv'):
    df_genus_log_drop08_scaled = pd.read_csv('../data/processed_genus_log_drop08_scaled.csv', sep='\t', header=0, index_col=0)
else:
    df_genus_log_drop08_scaled.to_csv('../data/processed_genus_log_drop08_scaled.csv', sep='\t', header=True, index=True)
